# Introduzione


# Librerie
Allo scopo di migliorare, risolvendo i bug incontrati, e potenziare i meccanismi per la risoluzione del 3D bin packing ho sviluppato la seguente libreria:

In [ ]:
import py3dbl

# Elementi per casi di studio
Per eseguire esperimenti su un caso realistico consideriamo un mezzo atto allo scopo della logistica dell'ultimo miglio, in particolare un Fiat Ducato:

In [ ]:
DucatoL2H2 = py3dbl.BinModel(name="DucatoL2H2",size=(1.87,1.932,3.120),max_weight=3000)

Generiamo una lista di ordini:

In [ ]:
orders = py3dbl.item_generator(
    width  = (0.05,1.5),
    height = (0.05,1.5),
    depth  = (0.05,1.5),
    weight = (10,1),
    priority_range = (0,10),
    batch_size   = 800,
    use_gaussian_distrib = False
)

In [ ]:
print(len([ o for o in orders if py3dbl.constraints['fits_inside_bin'](py3dbl.Bin(0,DucatoL2H2),o)]))

Configuriamo il risolutore (Packer)

In [ ]:
packer = py3dbl.Packer()
packer.set_default_bin(DucatoL2H2)
packer.add_batch(orders)

Lanciamo la risoluzione:

In [ ]:
import time

start = time.time()
packer.pack(constraints=[
    py3dbl.constraints['weight_within_limit'],
    py3dbl.constraints['fits_inside_bin'],
    py3dbl.constraints['no_overlap'],
    py3dbl.constraints['is_supported']
])
end = time.time()
print("Time: ",end - start)


Vediamo i risultati:

In [ ]:
print(len(packer.current_configuration))
print(len(packer.items))
acc = 0
for bin in packer.current_configuration:
    acc += len(bin.items)
print(acc)

In [ ]:
#for bin in packer.current_configuration:
#    py3dbl.render_bin_interactive(bin,border_width=1.0,transparency=.8)
py3dbl.render_bin_interactive(packer.current_configuration[0],render_bin=False,border_width=1.0,transparency=.8)